In [1]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings

In [2]:
jeju= pd.read_csv('C:/workspace/jeju/2020_jeju_creditcard/data/data_with_same_home_proportion.csv',encoding = 'EUC-KR',index_col=0)
jeju.head()

C:\Users\drlim\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT,month,...,released,deceased,YYMM,festival_count,rain_day,sex_age_population,screen_num,total_tourist,total_sale_by_indst,total_home
1,강원,건강보조식품 소매업,강원,20s,1,1,13,843100.000,16,11,...,0,0,2019-11-01,0,9.100,100115,28,1808903.949,33849.000,0.749
2,강원,건강보조식품 소매업,강원,20s,1,1,3,139000.000,3,6,...,0,0,2019-06-01,8,10.600,100115,28,1374767.001,33849.000,0.718
3,강원,건강보조식품 소매업,강원,20s,1,1,3,27500.000,3,8,...,0,0,2019-08-01,8,13.900,100115,28,1603894.835,33849.000,0.574
4,강원,건강보조식품 소매업,강원,20s,1,1,3,395500.000,3,9,...,0,0,2019-09-01,11,9.800,100115,28,2062150.502,33849.000,0.723
5,강원,건강보조식품 소매업,강원,20s,1,1,3,427510.000,2,3,...,23,1,2020-03-01,0,7.300,100115,28,nan,33849.000,0.816


In [3]:
jeju.shape

(1048575, 22)

In [4]:
region = pd.read_csv('C:/workspace/jeju/2020_jeju_creditcard/data/Region.csv',encoding = 'EUC-KR',index_col=0)

In [5]:
region.head()

,province,city,latitude,longitude,elementary_school_count,kindergarten_count,university_count,academy_ratio,elderly_population_ratio,elderly_alone_ratio,nursing_home_count
code,,,,,,,,,,,
10000,서울,서울,37.567,126.978,607,830,48,1.440,15.380,5.800,22739
10010,서울,강남구,37.518,127.047,33,38,0,4.180,13.170,4.300,3088
10020,서울,강동구,37.530,127.124,27,32,0,1.540,14.550,5.400,1023
10030,서울,강북구,37.640,127.026,14,21,0,0.670,19.490,8.500,628
10040,서울,강서구,37.551,126.850,36,56,1,1.170,14.390,5.700,1080


In [6]:
#column filter
region = region[['province','city','elementary_school_count','kindergarten_count','university_count','academy_ratio']]

In [7]:
#row filter
region = region[region.province == region.city]

In [8]:
region

,province,city,elementary_school_count,kindergarten_count,university_count,academy_ratio
code,,,,,,
10000,서울,서울,607,830,48,1.440
11000,부산,부산,304,408,22,1.400
12000,대구,대구,229,355,11,1.620
13000,광주,광주,155,312,17,2.380
14000,인천,인천,250,403,7,1.270
15000,대전,대전,148,260,15,1.490
16000,울산,울산,119,200,4,2.210
17000,세종,세종,48,60,3,1.780
20000,경기도,경기도,1277,2237,61,1.600


In [9]:
region.replace(['경기도','강원도','충청북도','충청남도','전라북도','전라남도','경상북도','경상남도','제주도'],['경기','강원','충북','충남','전북','전남','경북','경남','제주'],inplace=True)

In [10]:
region = region[region.province!='대한민국']

In [11]:
region=region.drop(['city'],axis=1)
region.rename(columns={'province':'CARD_SIDO_NM'},inplace=True)

In [13]:
merge = pd.merge(jeju, region, left_on=['CARD_SIDO_NM'],right_on=['CARD_SIDO_NM'],how='left')

In [15]:
merge.shape

(1048575, 26)

In [17]:
merge.to_csv('C:/workspace/jeju/2020_jeju_creditcard/data/final_merge_0718.csv',encoding = 'EUC-KR')